In [2]:
path = "../data/weather/midas-open_uk-hourly-weather-obs_dv-202507_avon_62122_almondsbury_qcv-1_2024.csv"

with open(path, "r", errors="ignore") as f:
    for i in range(5):
        line = f.readline()
        print(f"LINE {i+1}:", line[:200])

LINE 1: Conventions,G,BADC-CSV,1

LINE 2: title,G,uk-hourly-weather-obs

LINE 3: source,G,Met Office MIDAS database

LINE 4: creator,G,Met Office

LINE 5: activity,G,Met Office MIDAS Open: UK Land Surface Stations Data



In [4]:
path = "../data/weather/midas-open_uk-hourly-weather-obs_dv-202507_avon_62122_almondsbury_qcv-1_2024.csv"

start = None
with open(path, "r", errors="ignore") as f:
    for i, line in enumerate(f):
        if line.strip().lower() == "data":
            start = i + 1
            print("FOUND exact 'data' marker at line:", i)
            break

start

FOUND exact 'data' marker at line: 282


283

In [5]:
import pandas as pd

weather = pd.read_csv(path, skiprows=start)
print(weather.shape)
print(weather.columns.tolist()[:30])
weather.head()

(8778, 104)
['ob_time', 'id', 'id_type', 'met_domain_name', 'version_num', 'src_id', 'rec_st_ind', 'wind_speed_unit_id', 'src_opr_type', 'wind_direction', 'wind_speed', 'prst_wx_id', 'past_wx_id_1', 'past_wx_id_2', 'cld_ttl_amt_id', 'low_cld_type_id', 'med_cld_type_id', 'hi_cld_type_id', 'cld_base_amt_id', 'cld_base_ht', 'visibility', 'msl_pressure', 'cld_amt_id_1', 'cloud_type_id_1', 'cld_base_ht_id_1', 'cld_amt_id_2', 'cloud_type_id_2', 'cld_base_ht_id_2', 'cld_amt_id_3', 'cloud_type_id_3']


/var/folders/8h/f8h6db8n04z7dkdqqxfyt7jm0000gn/T/ipykernel_4104/3602942762.py:3: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv(path, skiprows=start)


,ob_time,id,id_type,met_domain_name,version_num,src_id,rec_st_ind,wind_speed_unit_id,src_opr_type,wind_direction,...,wetb_temp_j,rltv_hum_j,vert_vsby_j,stn_pres_j,alt_pres_j,q10mnt_mxgst_spd_j,meto_stmp_time,midas_stmp_etime,drv_hr_sun_dur,drv_hr_sun_dur_q
0,2024-01-01 00:00:00,3930.0,WMO,SYNOP,1.0,62122.0,1011.0,4.0,6.0,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01 00:02:13,NaN,0.0,1.0
1,2024-01-01 01:00:00,3930.0,WMO,SYNOP,1.0,62122.0,1011.0,4.0,6.0,240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01 01:01:11,NaN,0.0,1.0
2,2024-01-01 02:00:00,3930.0,WMO,SYNOP,1.0,62122.0,1011.0,4.0,6.0,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01 02:01:13,NaN,0.0,1.0
3,2024-01-01 03:00:00,3930.0,WMO,SYNOP,1.0,62122.0,1011.0,4.0,6.0,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01 03:01:11,NaN,0.0,1.0
4,2024-01-01 04:00:00,3930.0,WMO,SYNOP,1.0,62122.0,1011.0,4.0,6.0,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01 04:01:08,NaN,0.0,1.0


In [7]:
import numpy as np

# 1) Standardise column names (lowercase)
weather.columns = [c.strip().lower() for c in weather.columns]

print("Loaded weather shape:", weather.shape)
print("Columns:", weather.columns.tolist()[:40])

# 2) Find the datetime column (MIDAS often uses one of these)
time_candidates = ["ob_time", "obstime", "observation_time", "date_time", "datetime", "time"]
time_col = next((c for c in time_candidates if c in weather.columns), None)
if time_col is None:
    raise ValueError("Couldn't find observation time column. Paste weather.columns here and I'll adapt.")

# 3) Pick key weather variables (try common MIDAS names)
def pick_col(candidates):
    return next((c for c in candidates if c in weather.columns), None)

temp_col = pick_col(["air_temperature", "air_temp", "temperature", "temp", "t"])
rain_col = pick_col(["precipitation_amount", "precipitation", "rainfall_amount", "rainfall", "rain", "precip"])
wind_col = pick_col(["wind_speed", "windspeed", "wind_spd", "wind"])

print("Using columns:",
      {"time": time_col, "temp": temp_col, "rain": rain_col, "wind": wind_col})

# 4) Build tidy dataframe
cols = [c for c in [time_col, temp_col, rain_col, wind_col] if c is not None]
w = weather[cols].copy()

# Convert numeric fields
for c in [temp_col, rain_col, wind_col]:
    if c is not None:
        w[c] = pd.to_numeric(w[c], errors="coerce")

# 5) Parse datetime and convert UK local -> UTC
w["datetime"] = pd.to_datetime(w[time_col], errors="coerce")

# MIDAS hourly observations are typically UK local time
###
# Handle DST safely:
# - ambiguous="NaT" drops the repeated fall-back hour (01:00 occurs twice)
# - nonexistent="shift_forward" handles spring-forward missing hour
w["datetime"] = (
    w["datetime"]
      .dt.tz_localize("Europe/London", ambiguous="NaT", nonexistent="shift_forward")
      .dt.tz_convert("UTC")
)

# Drop rows where datetime became NaT
w = w.dropna(subset=["datetime"])

# Drop the original time column
w = w.drop(columns=[time_col])

# 6) Rename to clean names
rename_map = {}
if temp_col is not None: rename_map[temp_col] = "temp_c"
if rain_col is not None: rename_map[rain_col] = "rain_mm"
if wind_col is not None: rename_map[wind_col] = "wind_ms"

w = w.rename(columns=rename_map)

# 7) One row per hour (just in case duplicates exist)
w = (w.sort_values("datetime")
       .groupby("datetime", as_index=False)
       .agg({col: "mean" for col in w.columns if col != "datetime"}))

print("Clean weather rows:", len(w))
print(w.head())

# 8) Save clean file
OUT = "../data/weather/weather_2024_almondsbury_clean.csv"
w.to_csv(OUT, index=False)
print("Saved:", OUT)

Loaded weather shape: (8778, 104)
Columns: ['ob_time', 'id', 'id_type', 'met_domain_name', 'version_num', 'src_id', 'rec_st_ind', 'wind_speed_unit_id', 'src_opr_type', 'wind_direction', 'wind_speed', 'prst_wx_id', 'past_wx_id_1', 'past_wx_id_2', 'cld_ttl_amt_id', 'low_cld_type_id', 'med_cld_type_id', 'hi_cld_type_id', 'cld_base_amt_id', 'cld_base_ht', 'visibility', 'msl_pressure', 'cld_amt_id_1', 'cloud_type_id_1', 'cld_base_ht_id_1', 'cld_amt_id_2', 'cloud_type_id_2', 'cld_base_ht_id_2', 'cld_amt_id_3', 'cloud_type_id_3', 'cld_base_ht_id_3', 'cld_amt_id_4', 'cloud_type_id_4', 'cld_base_ht_id_4', 'vert_vsby', 'air_temperature', 'dewpoint', 'wetb_temp', 'rltv_hum', 'stn_pres']
Using columns: {'time': 'ob_time', 'temp': 'air_temperature', 'rain': None, 'wind': 'wind_speed'}
Clean weather rows: 8775
                   datetime  temp_c  wind_ms
0 2024-01-01 00:00:00+00:00     7.8     15.0
1 2024-01-01 01:00:00+00:00     8.0     14.0
2 2024-01-01 02:00:00+00:00     8.0     16.0
3 2024-01-01

In [8]:
print("Weather datetime range (UTC):", w["datetime"].min(), "to", w["datetime"].max())
print("Unique hours:", w["datetime"].nunique(), "Rows:", len(w))

Weather datetime range (UTC): 2024-01-01 00:00:00+00:00 to 2024-12-31 23:00:00+00:00
Unique hours: 8775 Rows: 8775


In [9]:
big = pd.read_csv("../data/mdm2_data_files/big_table_with_dark.csv")
big["datetime"] = pd.to_datetime(big["datetime"], utc=True, errors="coerce")

w = pd.read_csv("../data/weather/weather_2024_almondsbury_clean.csv")
w["datetime"] = pd.to_datetime(w["datetime"], utc=True, errors="coerce")

merged = big.merge(w, on="datetime", how="left")

print("Missing weather rates:")
print(merged[["temp_c","wind_ms"]].isna().mean())

Missing weather rates:
temp_c     0.003867
wind_ms    0.001333
dtype: float64


In [10]:
merged.to_csv("../data/mdm2_data_files/big_table_with_weather.csv", index=False)